<img src="Figurer/NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# ADC theory and continuous sampling

* **Emne AIS2201 - Signalbehandling**
* **Uke 36, 2024**
* **Underviser: Kai Erik Hoff**



# Topics
* Sample & Hold
* Successive Approximation Register ADC's
* ADC modes of operation

# Key points on peripherals

* The Analog-to-Digital converters on a microcontroller board are *peripheral* components
    * Independent components - can operate without using processor resources
    * Communication over data bus using *registers*
    * Operating speed derived from main processor clock signal

<img src="figures/ADC_peripheral_overview.png" style="width: 80%; margin-left: 100px" />

# Example: ADC illustration for STM32

<img src="figures/ADC_block_diagram.png" style="width: 80%; margin-left: 100px" />


* Analog multiplexer allows ADC to connect to many different ports
* Internal registers store configuration (which port to connect to etc.)
* Analog-to-digital conversion can be triggered by software
* Converted data stored in "output" register
* DMA request  / interrupt request sent when conversion is complete

# Using ADC's in "blocking mode" (a.k.a. `analogRead()`)



* While using tools such as `analogRead()` to acquire analog input on demand is simple, it is not particularly efficient.
    * Also requires implementing *software solutions* to achieve periodic sampling, instead of making use of hardware designed for the job.

* Sample acquisition steps in "blocking mode":
    1. Write new configuration to ADC registers (e.g. which port to connect to)
    2. Send software trigger to start A/D conversion
    3. Wait for conversion to complete
    4. Read sample information from ADC output register

* Additionally, functions such as `analogRead()` are *hardware agnostic*, meaning their use is the same across a wide range of microcontroller models. While this makes it easy for newcomers to program a microprocessor without difficulty, the consequence is that execution `analogRead()` has a significant overhead. As can be seen in the [analogRead source code](https://github.com/arduino/ArduinoCore-avr/blob/master/cores/arduino/wiring_analog.c#L38), a large portion of the function consists of a sizeable decision structure (many if-statements) which determines what configuration to use depending on the hardware platform.

# Using ADC's in "continuous mode"



* The port(s) we want an ADC to sample from only needs to be configured once at the start-up portion of the program
    * For use of multiple different ports the ADC can be configured to do a "sweep"

* The ADC can be set to begin a new A/D conversion immediately following a completed conversion
    * Notification can be sent to the main processor once a sample is ready using [interrupt requests](https://en.wikipedia.org/wiki/Interrupt_request).

* Sampling frequency is determined by total A/D conversion time.

# A/D conversion time

* Governed by peripheral clock frequency (derived from main system clock)

<img src="figures/clock_signal.jpg" style="width: 50%; margin-left: 100px" />

* A $180\text{ MHz}$ microprocessor runs on a clock signal with $180\ 000\ 000$ such pulses per second.
    * Processor completes one logigal operation per pulse
    * Peripheral clock frequency occasionally scaled down from main clock frequency (divided by $2$, $4$ ,$8$ etc..)
    * ADC has built-in prescaler to further reduce clock frequency

* One A/D conversion requires a set number of *clock cycles*
    * Determined by type of ADC and configuration

# Sample & hold

* First stage in the Analog-to-Digital conversion process
* Purpose: acquire the input voltage at a specific time ($n\cdot T_s$) and keep this voltage constant while conversion to digital information is underway.

<img src="figures/sample_hold.png" style="width: 70%; margin-left: 100px" />

* This approach is common for most ADC designs. The differences arise in *how* the binary representation is achieved using the sample & hold signal as input.
* From this point on we will focus on one specific type of ADC: the **Successive Approximation Register**. The reason for this choice is because it is this type of ADC we will encounter when programming our STM32 mircrocontroller.

## "Sample phase" illustrated

<img src="figures/sample_phase.png" style="width: 80%; margin-left: 100px" />

* The input voltage is fed to a capacitor, which will be able to hold a charge once the connection is broken. The logical circuitry which is responsible for the actual A/D conversion is idle during this time.

## "Hold phase" illustrated

<img src="figures/hold_phase.png" style="width: 80%; margin-left: 100px" />


* Connection between analog input and capacitor is broken. Assuming infinite input impedance in the successive approximation register, the capacitor will maintain it's charge for the duration of the conversion.

# The successive approximation register

<img src="figures/SAR_illustration.png" style="width: 80%; margin-left: 100px" />

* Converts *one bit at a time* starting with the most significant bit (msb)
* The DAC will produce a voltage output determined by data register content
* The comparator compares capacitor voltage and DAC output.
    * Outputs `1` if $v_C > v_{DAC}$
    * Outputs `0` if $v_C < v_{DAC}$
* The conversion process can be compared to a [*binary search*](https://en.wikipedia.org/wiki/Binary_search)

## SAR example conversion:   $\ \ v_c = 2.1\text{ V}, \ \ \text{n}_{\text{bits}}=6$


In [1]:
from Kildekode.image_cycler import ImageCycler
%matplotlib ipympl
ImageCycler(['SAR_ex1.png', 'SAR_ex2.png', 'SAR_ex3.png', 'SAR_ex4.png', 'SAR_ex5.png', 'SAR_ex6.png', 'SAR_ex7.png'],
            "Figurer/05_Sampling_og_Aliasing/",
            description = "Clock cycle nr.");

ModuleNotFoundError: No module named 'Kildekode'

# Determining the sampling rate for continuous sampling

#### Summary of factors which affect sampling frequency
* Successive approximation conversion time
    * Unit: clock cycles
    * Not configurable unless a reduction in bit precision is acceptable
* Sample time (*the duration of the "sample phase" where the capacitor is charged*)
    * Unit: clock cycles
    * Highly configurable
* ADC prescaler,
    * Determines relative speed of ADC clock ot peripheral clock
    * Integer value
    * Configurable with constraints (must be power of two $2^N$)
* Peripheral clock prescaler
    * Determines relative speed of peripheral clock to system clock
    * Integer value
    * Configurable with constraints (must be power of two $2^N$)
* System clock
    * Fundamental "heartbeat" of entire microcontroller
    * Unit: Hz/KHz/MHz
    * Partially configurable

# Continuous sample rate calculation

* Combining all factors above presents a formula for determining sampling frequency $f_s$:

## $$f_s = \frac{\frac{\text{System Clock}}{\text{Peripheral Prescaler }\times \text{ ADC Prescaler}}}{\text{Sample Time} + \text{Conversion Time}} $$

# ADC configuration
* On a microcontroller, registers are accessible for reading/writing using [macros](https://www.geeksforgeeks.org/macros-and-its-types-in-c-cpp/).
    * The names of these macros and what each bit represents is listed in the reference manual for the microcontroller in question.

#### Example:
```C++
ADC1->CR2 = <some code here>;
```
* Macro for selecting "Control Register 2" for "ADC 1" (stm32 has 3 ADCs)

# Accessing individual bits

* When modifying a register we often wish to set a specific bit in an integer to either `1` or `0` while at the same time leaving the rest of the bits untouched.

#### Example:

* add something here later!

* Requires bitwise operators

# Bitwise operators

| Operator | Function | Example |
|---|---|---|
|`~`| Bitwise NOT | `~0b00001111` equals `0b11110000` | 
|`&`| Bitwise AND | `0b00001111 & 0b00111100` equals `0b00001100`|
|`\|`| Bitwise OR | `0b00001111 & 0b00111100` equals `0b00111111`|
|`^`| Bitwise XOR | `0b00001111 & 0b00111100` equals `0b00110011`|
|`>>`|Bitwise Right Shift| `0b11110000 >> 1` equals `0b01111000`|
|`<<`|Bitwise Left Shift | `0b00001111 << 1` equals `0b00011110`| 

### Code example 1
* Set bit nr. 0 (counting from the right and starting with 0) to `1` in the variable `EX_REGISTER`

In [ ]:
EX_REGISTER = 0x8244 # Hexadecimal notation, 16 bit

# DO SOMETHING HERE!

print(f"{EX_REGISTER:b}")

### Code example 2
* Set bit nr. 5 (counting from the right and starting with 0) to `1` in the variable `EX_REGISTER`

In [ ]:
# DO SOMETHING HERE!

print(f"{EX_REGISTER:b}")

### Code example 3
* Set bit nr. 6 (counting from the right and starting with 0) to `0` in the variable `EX_REGISTER`

In [ ]:
# DO SOMETHING HERE!

print(f"{EX_REGISTER:b}")

### Code example 4
* Set bits 9 through 11 to `101` in the variable `EX_REGISTER`

In [ ]:
# DO SOMETHING HERE!

print(f"{EX_REGISTER:b}")

# Closing remarks

* A lot of hardware manufacturers provide libraries for interacting with hardware, so we can make use of the hardware without requiring us to figure out all the register settigns ourselves.
    * The STM32 CubeIDE lets us use a GUI tool to generate all the hardware-configuring code.
* It is still useful to have a conception of what goes on "underneath the hood". 
* We can be prepared for those occasions when such a library is not provided, or we wish to do something very specific not supported by the library.

# Questions?